In [1]:
!pip install -q tf-nightly
import tensorflow_datasets as tfds
import tensorflow as tf

     |████████████████████████████████| 517.7MB 26kB/s 
     |████████████████████████████████| 460kB 47.0MB/s 
     |████████████████████████████████| 3.0MB 40.0MB/s 


In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [0]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_examples, test_examples = dataset['train'], dataset['test']


In [0]:
encoder = info.features['text'].encoder

In [7]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [9]:
sample_string = 'Hello Python'
encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: {}'.format(original_string))

Encoded string is [4025, 222, 8009, 6002, 136]
The original string: Hello Python


In [0]:
assert original_string == sample_string

In [11]:
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
8009 ----> P
6002 ----> yth
136 ----> on


In [0]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64


In [0]:
train_dataset = (train_examples
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE))

test_dataset = (test_examples
                .padded_batch(BATCH_SIZE))

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(encoder.vocab_size,64),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [0]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [17]:
history = model.fit(train_dataset, epochs=10,
                    validation_data = test_dataset,
                    validation_steps=30)


Epoch 1/10
391/391 [==============================] - 878s 2s/step - loss: 0.6547 - accuracy: 0.5466 - val_loss: 0.5460 - val_accuracy: 0.7318
Epoch 2/10
391/391 [==============================] - 880s 2s/step - loss: 0.3905 - accuracy: 0.8290 - val_loss: 0.3440 - val_accuracy: 0.8599
Epoch 3/10
391/391 [==============================] - 880s 2s/step - loss: 0.2559 - accuracy: 0.8973 - val_loss: 0.3257 - val_accuracy: 0.8755
Epoch 4/10
391/391 [==============================] - 879s 2s/step - loss: 0.2109 - accuracy: 0.9186 - val_loss: 0.3479 - val_accuracy: 0.8760
Epoch 5/10
391/391 [==============================] - 887s 2s/step - loss: 0.1820 - accuracy: 0.9316 - val_loss: 0.3361 - val_accuracy: 0.8641
Epoch 6/10
391/391 [==============================] - 893s 2s/step - loss: 0.1630 - accuracy: 0.9407 - val_loss: 0.3766 - val_accuracy: 0.8724
Epoch 7/10
391/391 [==============================] - 888s 2s/step - loss: 0.1470 - accuracy: 0.9482 - val_loss: 0.3811 - val_accuracy: 0.8661

In [0]:
test_loss, test_acc = model.avaluate(test_dataset)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [0]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [0]:
def sample_predict(sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tfcast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [0]:
sample_pred_text = ('The movie was cool. The animation and the graphics' 'were out of this world. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad = False)
print(predictions)

In [0]:
# predict on a sample text with padding

sample_pred_text = ('The movie was cool. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)

In [0]:
plot_graphs(history, 'accuracy')